## Classification of Benthic Megafauna using a CNN

Before you can use this notebook, make sure that you have downloaded the basml.py (python) file and put it into your working directory. Also, make sure that you have all of the required libraries installed; libraries can be installed using:


pip install -r requirements.txt

Alternatively, you can git clone the whole repository https://github.com/brett-hosking/BASML_DataChallenge.git

See https://github.com/brett-hosking/BASML_DataChallenge for more details

Download and extract the data: 
'https://github.com/brett-hosking/BASML_DataChallenge/blob/master/data256.zip?raw=true'

Download basml.py file:
'https://github.com/brett-hosking/BASML_DataChallenge/blob/master/basml.py?raw=true'

Download requirements.txt file:
'https://github.com/brett-hosking/BASML_DataChallenge/blob/master/requirements.txt?raw=true'

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
import basml # some pre-made functions for this data challenge
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
%matplotlib inline

Load data using pre-made function in the basml mini library

In [2]:
X,Y = basml.loaddata('data256') # the folder containing the downloaded data for this challenege
print('data:', np.shape(X))
print('labels:', np.shape(Y))

data: (2094, 256, 256, 3)
labels: (2094, 4)


normalise the values, in range [0,1] 

In [3]:
X /= 255.0 


Generate a random training and test. Here we use 80% of the data for training and 20% for test

In [4]:
X,Y = basml.randomiseXY(X,Y)
Xtrain,Ytrain,Xtest,Ytest = basml.ttsplit(X,Y,per=20)

In [5]:
print(np.shape(Xtrain),np.shape(Ytrain), np.shape(Xtest), np.shape(Ytest))

(1675, 256, 256, 3) (1675, 4) (419, 256, 256, 3) (419, 4)


Apply zeromean normalisation

In [6]:
zeromean = np.mean(Xtrain)
Xtrain -=zeromean
Xtest -=zeromean

Build the CNN model

In [10]:
model = keras.Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(256,256,3)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Train the CNN using the training data (using the validation_data hyperparameter is optional)

In [53]:
model.fit(Xtrain, Ytrain, epochs=5,validation_data=(Xtest, Ytest))


Train on 1675 samples, validate on 419 samples
Epoch 1/5
1675/1675 [==============================] - 247s 148ms/sample - loss: 0.6118 - acc: 0.8496 - val_loss: 0.7657 - val_acc: 0.7494
Epoch 2/5
1675/1675 [==============================] - 287s 171ms/sample - loss: 0.4534 - acc: 0.8854 - val_loss: 0.5352 - val_acc: 0.8473
Epoch 3/5
1675/1675 [==============================] - 397s 237ms/sample - loss: 0.3280 - acc: 0.9296 - val_loss: 0.4009 - val_acc: 0.8998
Epoch 4/5
1675/1675 [==============================] - 244s 146ms/sample - loss: 0.1914 - acc: 0.9504 - val_loss: 0.4469 - val_acc: 0.8783
Epoch 5/5
1675/1675 [==============================] - 247s 148ms/sample - loss: 0.1800 - acc: 0.9606 - val_loss: 0.4781 - val_acc: 0.8902


Use the model to predict the class of samples in the test set and reformat as one-hot, e.g. [0,1,0,0] for the second class

In [54]:
testlen = len(Ytest)
Ypred= np.array(np.zeros((testlen,4)))
predictions = model.predict(Xtest)
for i in range(testlen):
    Ypred[i][np.argmax(predictions[i])] 	= 1

Use scikit-learn to generate a report

In [55]:
report = classification_report(Ytest, Ypred)#, target_names=class_list,digits=6,labels=range(classes))
print(report)

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       101
           1       0.86      0.93      0.89       104
           2       0.98      0.77      0.86       113
           3       0.81      0.91      0.86       101

   micro avg       0.89      0.89      0.89       419
   macro avg       0.90      0.89      0.89       419
weighted avg       0.90      0.89      0.89       419
 samples avg       0.89      0.89      0.89       419

